In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json

# ChromeDriver 경로 설정 및 옵션
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 매일경제 검색 페이지로 이동
base_url = "https://www.mk.co.kr/search?word=%EC%97%94%ED%99%94"
driver.get(base_url)

# 페이지 로드 대기
time.sleep(3)

# '더보기' 버튼 두 번 클릭하기
try:
    for _ in range(4):  # 두 번 클릭
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "drop_sub_news_btn")]'))
        )
        driver.execute_script("arguments[0].click();", more_button)
        
        # 새로운 뉴스가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="news_node"]'))
        )
        
        time.sleep(3)  # 추가적인 페이지 로드 시간 확보
except Exception as e:
    print("더보기 버튼을 클릭하는데 실패했습니다:", e)

# '더보기' 버튼을 클릭한 후, 모든 뉴스 항목을 크롤링 시작
soup = BeautifulSoup(driver.page_source, 'html.parser')
news_items = soup.find_all('li', {'class': 'news_node'})

# 뉴스 데이터를 저장할 리스트
news_data = []

# 뉴스 기사 링크 수집
news_links = []
for item in news_items:
    link = item.find('a', {'class': 'news_item'}).get('href')
    # URL 형식 확인 및 수집
    if not link.startswith("https://www.mk.co.kr/economy/view/") :
        news_links.append(link)

# 각 뉴스 페이지에서 정보 크롤링
for link in news_links:
    # URL 형식이 올바른지 확인
    if not link.startswith("http"):
        print(f"Invalid URL skipped: {link}")
        continue

    try:
        driver.get(link)
    except Exception as e:
        print(f"Failed to load the page {link}: {e}")
        continue

    time.sleep(2)  # 페이지 로드 대기

    # 뉴스 페이지 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # HTML 구조 확인 (title, image, content)
    title_tag = soup.find('h2', {'class': 'news_ttl'})
    image_div = soup.find('div', {'class': 'thumb'})
    content_div = soup.find('div', {'class': 'news_cnt_detail_wrap'})

    # HTML 구조가 명시한 구조가 아닌 경우, 다음 뉴스로 넘어가기
    if not title_tag or not content_div:
        continue

    # 데이터 추출
    title = title_tag.text.strip()
    content = ' '.join([p.text.strip() for p in content_div.find_all('p')])
    image_url = image_div.find('img')['src'] if image_div else None

    # content가 비어 있지 않을 때만 저장
    if content:
        news_data.append({
            'title': title,
            'content': content,
            'image_url': image_url,
            'url': link
        })
    print(len(news_data))
    # 수집한 뉴스 데이터가 30개면 중단
    if len(news_data) >= 30:
        break

# 결과 출력
for news in news_data[:30]:  # 상위 10개만 출력
    print(f"Title: {news['title']}")
    print(f"Content: {news['content']}")
    print(f"Image URL: {news['image_url']}")
    print(f"Article URL: {news['url']}")
    print('-' * 80)
# 브라우저 종료
driver.quit()

# JSON 파일로 저장
with open('news_data_10000_JPY.json', 'w', encoding='utf-8') as json_file:
    json.dump(news_data, json_file, ensure_ascii=False, indent=4)

print("News data has been saved to news_data_10000.json")

0
0
0
1
2
2
2
3
4
5
5
6
7
7
8
9
10
11
11
11
11
12
13
14
15
16
16
16
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
Title: 美 대선 D-66 혼돈의 금융시장 긴급점검
Content: 국내외 금융시장이 각종 돌발 변수로 인해 그야말로 롤러코스터를 타고 있다. 역대 최대 수준으로 급락했다가 곧장 사상 최고 수준으로 치솟는 등 투자자들의 혼을 쏙 빼놓고 있다. 지난 8월 초 미국발(發) 경기 침체 공포로 시작한 코스피 급등락이 엔캐리 트레이드 청산(저렴한 엔화로 매수한 해외 자산 재매도), 불안한 중동 정세, 인공지능(AI) 거대 기업 실적 우려에 특히 미국 대선 불확실성에도 영향을 받으면서 멈추지 않는 분위기다. 증시 전문가 대다수는 지금과 같이 변동성이 큰 장세가 한동안 이어질 것으로 본다. 문제는 이 상황이 언제 끝날지 아무도 모른다는 점이다. 누군가는 그 시점을 미국연방준비제도(Fed·연준) 연방공개시장위원회(FOMC)가 9월 ‘빅컷(0.5%포인트 이상 기준금리 인하)’을 단행할 때로, 누군가는 11월 미국 대선이 끝날 때로 예상하기도 한다.  가장 큰 변수는 예측이 불가한 미국 대선이다. 후보자인 트럼프와 해리스의 정치적 색채가 워낙 뚜렷하게 구분되다 보니 대선 결과에 따라 수혜 업종이 확연히 갈린다. 이 때문에 7월 트럼프 후보 피격 사건 이후엔 ‘트럼프 트레이드’가 부각되다 7월 말 해리스 부대통령의 대통령 후보 지명 이후엔 소강 상태에 접어들었다. 양 후보의 정책 기본 철학을 보면 트럼프는 세율을 낮춰 기업 친화적 환경을 조성하고 전통 제조업을 살려 많은 일자리 창출을 유도한다. 외교적으로는 미국 우선주의를 강조하여 실리 중심 외교를 선호한다. 2017년 트럼프 집권 당시 지수 수익률 및 매크로 지표들을 보면 트럼프의 생각이 구체적으로 어떻게 적용되었는지를 보여준다. 당시 주가 상승이 계속됐던 화석 연료 에너지, 원자력 산업, 전통 제조업, 방위 산업, 금융 산

News data has been saved to news_data_10000.json


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json

# ChromeDriver 경로 설정 및 옵션
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 매일경제 검색 페이지로 이동
base_url = "https://www.mk.co.kr/search?word=%EC%9C%84%EC%95%88%ED%99%94"
driver.get(base_url)

# 페이지 로드 대기
time.sleep(3)

# '더보기' 버튼 두 번 클릭하기
try:
    for _ in range(4):  # 두 번 클릭
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "drop_sub_news_btn")]'))
        )
        driver.execute_script("arguments[0].click();", more_button)
        
        # 새로운 뉴스가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="news_node"]'))
        )
        
        time.sleep(3)  # 추가적인 페이지 로드 시간 확보
except Exception as e:
    print("더보기 버튼을 클릭하는데 실패했습니다:", e)

# '더보기' 버튼을 클릭한 후, 모든 뉴스 항목을 크롤링 시작
soup = BeautifulSoup(driver.page_source, 'html.parser')
news_items = soup.find_all('li', {'class': 'news_node'})

# 뉴스 데이터를 저장할 리스트
news_data = []

# 뉴스 기사 링크 수집
news_links = []
for item in news_items:
    link = item.find('a', {'class': 'news_item'}).get('href')
    # URL 형식 확인 및 수집
    if not link.startswith("https://www.mk.co.kr/economy/view/") :
        news_links.append(link)

# 각 뉴스 페이지에서 정보 크롤링
for link in news_links:
    # URL 형식이 올바른지 확인
    if not link.startswith("http"):
        print(f"Invalid URL skipped: {link}")
        continue

    try:
        driver.get(link)
    except Exception as e:
        print(f"Failed to load the page {link}: {e}")
        continue

    time.sleep(2)  # 페이지 로드 대기

    # 뉴스 페이지 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # HTML 구조 확인 (title, image, content)
    title_tag = soup.find('h2', {'class': 'news_ttl'})
    image_div = soup.find('div', {'class': 'thumb'})
    content_div = soup.find('div', {'class': 'news_cnt_detail_wrap'})

    # HTML 구조가 명시한 구조가 아닌 경우, 다음 뉴스로 넘어가기
    if not title_tag or not content_div:
        continue

    # 데이터 추출
    title = title_tag.text.strip()
    content = ' '.join([p.text.strip() for p in content_div.find_all('p')])
    image_url = image_div.find('img')['src'] if image_div else None

    # content가 비어 있지 않을 때만 저장
    if content:
        news_data.append({
            'title': title,
            'content': content,
            'image_url': image_url,
            'url': link
        })
    print(len(news_data))
    # 수집한 뉴스 데이터가 30개면 중단
    if len(news_data) >= 30:
        break

# 결과 출력
for news in news_data[:30]:  # 상위 10개만 출력
    print(f"Title: {news['title']}")
    print(f"Content: {news['content']}")
    print(f"Image URL: {news['image_url']}")
    print(f"Article URL: {news['url']}")
    print('-' * 80)
# 브라우저 종료
driver.quit()

# JSON 파일로 저장
with open('news_data_20_CNY.json', 'w', encoding='utf-8') as json_file:
    json.dump(news_data, json_file, ensure_ascii=False, indent=4)

print("News data has been saved to news_data_CNY.json")

1
1
2
2
3
4
5
5
6
7
8
8
9
9
9
10
10
11
12
13
14
15
15
16
17
18
18
19
20
20
21
22
22
23
24
25
25
26
27
27
28
28
28
29
29
30
Title: “뭘 해도 이건 하지마”…학을 뗀 중학개미들, 수익률 얼마나 처참하길래
Content: 중국 증시가 지난 4월 깜짝 반등 이후 하락세를 지속하면서 국내 투자자들의 자금이 중국 주식형 펀드에서 급속도로 빠져나가고 있다. 지난 6개월간 빠져나간 금액이 1조원을 웃돌 정도다. 미국, 인도, 일본 등 최근 자금이 빠르게 유입되고 있는 여타 해외주식형펀드와 정반대 되는 모습이다. 중국 증시의 발목을 잡는 것은 경기 둔화 우려와 이에 따른 대표 기업들의 부진한 실적 전망이다. 살아나지 못하는 부동산시장을 비롯해 중국 경제의 구조적인 문제 탓에 중국 증시 회복이 쉽지 않을 것이란 전망이 나오고 있다. 12일 펀드평가사 에프앤가이드에 따르면 중국 주식형 펀드에서는 지난 10일 기준으로 최근 6개월간 1조871억원의 자금이 빠져나갔다. 중국 다음으로 유출이 컸던 건 802억원이 빠져나간 베트남 주식형 펀드인 걸 감안하면 해외 주식형 펀드중 단연 압도적인 수치다. 중국 증시 하락추세가 장기간 지속된 영향이 컸다는 분석이다. 중국 본토 대표 주가지수인 상하이종합지수는 올해들어 8.53% 하락했다. 중국정부가 올해 초 약 2조위안(379조원) 규모의 증권시장안정기금(증안기금) 투입 계획을 발표하고 ‘중국판 밸류업’으로 불리는 ‘신(新)국9조’ 등의 증시부양책을 발표하면서 잠시 반등했던 지난 5월 고점 대비해선 14.19%나 하락했다. 해외주식형 펀드 중에서 1·3·6개월·연초 이후 수익률이 모두 마이너스인 것도 중국 주식형 펀드가 유일하다. 시장 불황이 길어지며 국내 상장된 40여 개 중국 상장지수펀드(ETF)의 수익률도 악화되고 있다. 지난 6개월간 중국 본토 태양광 밸류체인 기업에 투자하는 상품인 ‘SOL 차이나태양광CSI(합성)’은 27.60% 하

News data has been saved to news_data_CNY.json


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

import mysql.connector
# ChromeDriver 경로 설정 및 옵션
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 매일경제 검색 페이지로 이동
base_url = "https://www.mk.co.kr/search?word=%EC%9C%84%EC%95%88%ED%99%94"
driver.get(base_url)

# 페이지 로드 대기
time.sleep(3)

# '더보기' 버튼 두 번 클릭하기
try:
    for _ in range(4):  # 두 번 클릭
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "drop_sub_news_btn")]'))
        )
        driver.execute_script("arguments[0].click();", more_button)
        
        # 새로운 뉴스가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="news_node"]'))
        )
        
        time.sleep(3)  # 추가적인 페이지 로드 시간 확보
except Exception as e:
    print("더보기 버튼을 클릭하는데 실패했습니다:", e)

# '더보기' 버튼을 클릭한 후, 모든 뉴스 항목을 크롤링 시작
soup = BeautifulSoup(driver.page_source, 'html.parser')
news_items = soup.find_all('li', {'class': 'news_node'})

# 뉴스 데이터를 저장할 리스트
news_data = []

# 뉴스 기사 링크 수집
news_links = []
for item in news_items:
    link = item.find('a', {'class': 'news_item'}).get('href')
    # URL 형식 확인 및 수집
    if not link.startswith("https://www.mk.co.kr/economy/view/") :
        news_links.append(link)

# 각 뉴스 페이지에서 정보 크롤링
for link in news_links:
    # URL 형식이 올바른지 확인
    if not link.startswith("http"):
        print(f"Invalid URL skipped: {link}")
        continue

    try:
        driver.get(link)
    except Exception as e:
        print(f"Failed to load the page {link}: {e}")
        continue

    time.sleep(2)  # 페이지 로드 대기

    # 뉴스 페이지 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # HTML 구조 확인 (title, image, content)
    title_tag = soup.find('h2', {'class': 'news_ttl'})
    image_div = soup.find('div', {'class': 'thumb'})
    content_div = soup.find('div', {'class': 'news_cnt_detail_wrap'})

    # HTML 구조가 명시한 구조가 아닌 경우, 다음 뉴스로 넘어가기
    if not title_tag or not content_div:
        continue

    # 데이터 추출
    title = title_tag.text.strip()
    content = ' '.join([p.text.strip() for p in content_div.find_all('p')])
    image_url = image_div.find('img')['src'] if image_div else None

    # content가 비어 있지 않을 때만 저장
    if content:
        news_data.append({
            'title': title,
            'content': content,
            'image_url': image_url,
            'url': link
        })
    print(len(news_data))
    # 수집한 뉴스 데이터가 30개면 중단
    if len(news_data) >= 30:
        break

# 결과 출력
for news in news_data[:30]:  # 상위 10개만 출력
    print(f"Title: {news['title']}")
    print(f"Content: {news['content']}")
    print(f"Image URL: {news['image_url']}")
    print(f"Article URL: {news['url']}")
    print('-' * 80)
# 브라우저 종료
driver.quit()


# MySQL 연결 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}


print("News data has been saved to news_data_CNY.json")

In [5]:
# 달러환율기사 클롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json

# ChromeDriver 경로 설정 및 옵션
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 매일경제 검색 페이지로 이동
base_url = "https://www.mk.co.kr/search?word=%EB%8B%AC%EB%9F%AC%ED%99%98%EC%9C%A8"
driver.get(base_url)

# 페이지 로드 대기
time.sleep(3)

# '더보기' 버튼 두 번 클릭하기
try:
    for _ in range(10):  # 두 번 클릭
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "drop_sub_news_btn")]'))
        )
        driver.execute_script("arguments[0].click();", more_button)
        
        # 새로운 뉴스가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="news_node"]'))
        )
        
        time.sleep(3)  # 추가적인 페이지 로드 시간 확보
except Exception as e:
    print("더보기 버튼을 클릭하는데 실패했습니다:", e)

# '더보기' 버튼을 클릭한 후, 모든 뉴스 항목을 크롤링 시작
soup = BeautifulSoup(driver.page_source, 'html.parser')
news_items = soup.find_all('li', {'class': 'news_node'})

# 뉴스 데이터를 저장할 리스트
news_data = []

# 뉴스 기사 링크 수집
news_links = set()
for item in news_items:
    link = item.find('a', {'class': 'news_item'}).get('href')
    # URL 형식 확인 및 수집
    if not link.startswith("https://www.mk.co.kr/economy/view/") :
        news_links.add(link)

        
news_links = list(news_links)

# 각 뉴스 페이지에서 정보 크롤링
for link in news_links:
    # URL 형식이 올바른지 확인
    if not link.startswith("http"):
        print(f"Invalid URL skipped: {link}")
        continue

    try:
        driver.get(link)
    except Exception as e:
        print(f"Failed to load the page {link}: {e}")
        continue

    time.sleep(2)  # 페이지 로드 대기

    # 뉴스 페이지 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # HTML 구조 확인 (title, image, content)
    title_tag = soup.find('h2', {'class': 'news_ttl'})
    image_div = soup.find('div', {'class': 'thumb'})
    content_div = soup.find('div', {'class': 'news_cnt_detail_wrap'})

    # HTML 구조가 명시한 구조가 아닌 경우, 다음 뉴스로 넘어가기
    if not title_tag or not content_div:
        continue

    # 데이터 추출
    title = title_tag.text.strip()
    content = ' '.join([p.text.strip() for p in content_div.find_all('p')])
    image_url = image_div.find('img')['src'] if image_div else None

    # content가 비어 있지 않을 때만 저장
    if content:
        news_data.append({
            'title': title,
            'content': content,
            'image_url': image_url,
            'url': link
        })
    print(len(news_data))
    # 수집한 뉴스 데이터가 50개면 중단
    if len(news_data) >= 100:
        break

# 결과 출력
for news in news_data[:30]:  # 상위 10개만 출력
    print(f"Title: {news['title']}")
    print(f"Content: {news['content']}")
    print(f"Image URL: {news['image_url']}")
    print(f"Article URL: {news['url']}")
    print('-' * 80)
# 브라우저 종료
driver.quit()

# JSON 파일로 저장
with open('recent_usd_news200.json', 'w', encoding='utf-8') as json_file:
    json.dump(news_data, json_file, ensure_ascii=False, indent=4)

print("News data has been saved to recent_usd_news.json.json")

1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 

In [8]:
# 달러환율기사 크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import json

# ChromeDriver 경로 설정 및 옵션
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# 페이지 로드 타임아웃 설정 (예: 5초)
driver.set_page_load_timeout(5)

# 매일경제 검색 페이지로 이동
base_url = "https://www.mk.co.kr/search?word=%EB%8B%AC%EB%9F%AC%ED%99%98%EC%9C%A8"
driver.get(base_url)

# 페이지 로드 대기
time.sleep(3)

# '더보기' 버튼 클릭 (원하는 만큼 조정)
try:
    for _ in range(20):
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "drop_sub_news_btn")]'))
        )
        driver.execute_script("arguments[0].click();", more_button)
        
        # 새로운 뉴스가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="news_node"]'))
        )
        
        time.sleep(3)  # 추가적인 페이지 로드 시간 확보
except Exception as e:
    print("더보기 버튼을 클릭하는데 실패했습니다:", e)

# '더보기' 버튼을 클릭한 후, 모든 뉴스 항목을 크롤링 시작
soup = BeautifulSoup(driver.page_source, 'html.parser')
news_items = soup.find_all('li', {'class': 'news_node'})

# 뉴스 데이터를 저장할 리스트
news_data = []

# 뉴스 기사 링크 수집
news_links = set()
for item in news_items:
    link = item.find('a', {'class': 'news_item'}).get('href')
    # URL 형식 확인 및 수집
    if not link.startswith("https://www.mk.co.kr/economy/view/") :
        news_links.add(link)

news_links = list(news_links)

# 각 뉴스 페이지에서 정보 크롤링
for link in news_links:
    # URL 형식이 올바른지 확인
    if not link.startswith("http"):
        print(f"Invalid URL skipped: {link}")
        continue

    try:
        # 페이지 로드 타임아웃 설정 (예: 5초)
        driver.set_page_load_timeout(5)
        driver.get(link)
    except TimeoutException:
        print(f"Timeout occurred while loading page: {link}")
        continue
    except Exception as e:
        print(f"Failed to load the page {link}: {e}")
        continue

    time.sleep(2)  # 페이지 로드 대기

    # 뉴스 페이지 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # HTML 구조 확인 (title, image, content)
    title_tag = soup.find('h2', {'class': 'news_ttl'})
    image_div = soup.find('div', {'class': 'thumb'})
    content_div = soup.find('div', {'class': 'news_cnt_detail_wrap'})

    # HTML 구조가 명시한 구조가 아닌 경우, 다음 뉴스로 넘어가기
    if not title_tag or not content_div:
        continue

    # 데이터 추출
    title = title_tag.text.strip()
    content = ' '.join([p.text.strip() for p in content_div.find_all('p')])
    image_url = image_div.find('img')['src'] if image_div else None

    # content가 비어 있지 않을 때만 저장
    if content:
        news_data.append({
            'title': title,
            'content': content,
            'image_url': image_url,
            'url': link
        })
    print(len(news_data))
    # 수집한 뉴스 데이터가 100개면 중단
    if len(news_data) >= 100:
        break

# 결과 출력
for news in news_data[:30]:  # 상위 30개만 출력
    print(f"Title: {news['title']}")
    print(f"Content: {news['content']}")
    print(f"Image URL: {news['image_url']}")
    print(f"Article URL: {news['url']}")
    print('-' * 80)

# 브라우저 종료
driver.quit()

# JSON 파일로 저장
with open('recent_usd_news200.json', 'w', encoding='utf-8') as json_file:
    json.dump(news_data, json_file, ensure_ascii=False, indent=4)

print("News data has been saved to recent_usd_news200.json")

1
2
3
4
4
5
6
7
8
9
10
11
11
12
13
14
14
15
15
16
17
18
19
20
20
21
21
22
23
23
24
25
25
25
26
26
26
26
27
28
29
30
31
32
33
34
35
36
37
37
38
39
39
40
40
41
41
41
42
43
43
43
44
45
45
46
47
48
48
49
Timeout occurred while loading page: https://www.mk.co.kr/news/stock/11074587
50
50
51
52
53
54
54
55
56
57
58
58
59
60
60
61
62
63
64
64
65
66
66
67
67
68
69
70
70
71
72
73
74
74
75
75
76
77
78
79
80
81
82
82
83
84
85
86
87
88
89
89
89
89
90
91
91
92
93
93
93
94
95
96
97
98
98
98
99
100
Title: “HD현대일렉트릭, 2분기 깜짝실적에 견조한 시황”…목표가 50%↑
Content: 하나증권은 24일 HD현대일렉트릭에 대해 미국의 정책 변화 가능성을 감안해도 시황이 견조하다며 목표주가를 기존 28만원에서 42만원으로 상향 조정했다. HD현대일렉트릭의 올해 2분기 실적은 시장 컨센서스(예상치)를 크게 상회했다. 전날 HD현대일렉트릭은 올 2분기 매출액 9169억원, 영억이익 2100억원을 기록했다고 공시했다. 전년동기대비 매출액은 42.7%, 영업이익은 257.1% 증가한 수준이다. 전년대비 모든 부문에서 외형이 성장했으며 전분기 대비로는 회전기기 부문에서만 소폭 역성장이 나타났다. 물류 이슈에 대응한 조기 납품에 더해 환율 영향도 마진에 긍정적으로 영향을 미친 것으로 해석했다. 일회성 이익으로 볼 수 있는 부분이 제한적이었기 때문에 매출 규모 확대에 따른 고정비 절감으로 연간 20%대 마진 달성 가능성은 충분하다는 게 하나증권의 설명이다. 2분기 수주액은 8억 8000만달러로 연간 수주

News data has been saved to recent_usd_news200.json
